# TensorFlow卷积神经网络CNN

In [24]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelBinarizer
from sklearn.cross_validation import train_test_split

In [25]:
digits = load_digits()
X = digits.data.astype('float32')
y = digits.target
y = LabelBinarizer().fit_transform(y)
X = X / 255
print X.shape
print y.shape
print X[0]

(1797, 64)
(1797, 10)
[ 0.          0.          0.01960784  0.05098039  0.03529412  0.00392157
  0.          0.          0.          0.          0.05098039  0.05882353
  0.03921569  0.05882353  0.01960784  0.          0.          0.01176471
  0.05882353  0.00784314  0.          0.04313726  0.03137255  0.          0.
  0.01568628  0.04705882  0.          0.          0.03137255  0.03137255
  0.          0.          0.01960784  0.03137255  0.          0.
  0.03529412  0.03137255  0.          0.          0.01568628  0.04313726
  0.          0.00392157  0.04705882  0.02745098  0.          0.
  0.00784314  0.05490196  0.01960784  0.03921569  0.04705882  0.          0.
  0.          0.          0.02352941  0.05098039  0.03921569  0.          0.
  0.        ]


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(1257, 64) (1257, 10)
(540, 64) (540, 10)


接着呢，我们定义Weight变量，输入shape，返回变量的参数。其中我们使用tf.truncted_normal产生随机变量来进行初始化:

In [27]:
def weight_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1) # 均值为0，方差为0.1的正态分布
    return tf.Variable(inital)

同样的定义biase变量，输入shape ,返回变量的一些参数。其中我们使用tf.constant常量函数来进行初始化:

In [28]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

定义卷积，`tf.nn.conv2d`函数是tensoflow里面的二维的卷积函数，x是图片的所有参数，W是此卷积层的权重，然后定义步长strides=[1,1,1,1]值，strides[0]和strides[3]的两个1是默认值，中间两个1代表padding时在x方向运动一步，y方向运动一步，padding采用的方式是SAME。

In [29]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

接着定义池化pooling，为了得到更多的图片信息，padding时我们选的是一次一步，也就是strides[1]=strides[2]=1，这样得到的图片尺寸没有变化，而我们希望压缩一下图片也就是参数能少一些从而减小系统的复杂度，因此我们采用pooling来稀疏化参数，也就是卷积神经网络中所谓的下采样层。pooling 有两种，一种是最大值池化，一种是平均值池化，本例采用的是最大值池化tf.max_pool()。池化的核函数大小为2x2，因此ksize=[1,2,2,1]，步长为2，因此strides=[1,2,2,1]:

In [30]:
def max_poo_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

这一次我们一层层的加上了不同的 layer. 分别是:
- convolutional layer1 + max pooling;
- convolutional layer2 + max pooling;
- fully connected layer1 + dropout;
- fully connected layer2 to prediction.

In [31]:
xs = tf.placeholder(tf.float32, [None, 64])
ys = tf.placeholder(tf.float32, [None, 10])

我们还定义了dropout的placeholder，它是解决过拟合的有效手段

In [32]:
keep_prob = tf.placeholder(tf.float32)

接着呢，我们需要处理我们的xs，把xs的形状变成[-1,8,8,1]，-1代表先不考虑输入的图片例子多少这个维度，后面的1是channel的数量，因为我们输入的图片是黑白的，因此channel是1，例如如果是RGB图像，那么channel就是3。

In [33]:
x_image = tf.reshape(xs, [-1, 8, 8, 1])

接着我们定义第一层卷积,先定义本层的Weight,本层我们的卷积核patch的大小是5x5，因为黑白图片channel是1所以输入是1，输出是32个featuremap



In [34]:
W_conv1 = weight_variable([5, 5, 1, 32])

接着定义bias，它的大小是32个长度，因此我们传入它的shape为[32]

In [35]:
b_conv1 = bias_variable([32])

定义好了Weight和bias，我们就可以定义卷积神经网络的第一个卷积层h_conv1=conv2d(x_image,W_conv1)+b_conv1,同时我们对h_conv1进行非线性处理，也就是激活函数来处理喽，这里我们用的是tf.nn.relu（修正线性单元）来处理，要注意的是，因为采用了SAME的padding方式，输出图片的大小没有变化依然是8x8，只是厚度变厚了，因此现在的输出大小就变成了8x8x32

In [36]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)

最后我们再进行pooling的处理就ok啦，经过pooling的处理，输出大小就变为了7x7x32

In [37]:
h_pool = max_poo_2x2(h_conv1)

接着呢，同样的形式我们定义第二层卷积，本层我们的输入就是上一层的输出，本层我们的卷积核patch的大小是5x5，有32个featuremap所以输入就是32，输出呢我们定为64

In [38]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

接着我们就可以定义卷积神经网络的第二个卷积层，这时的输出的大小就是7x7x64

In [39]:
h_conv2 = tf.nn.relu(conv2d(h_pool, W_conv2)+b_conv2)

最后也是一个pooling处理，输出大小为3x3x64

In [40]:
h_pool2 = max_poo_2x2(h_conv2)

好的，接下来我们定义我们的 fully connected layer,

进入全连接层时, 我们通过tf.reshape()将h_pool2的输出值从一个三维的变为一维的数据, -1表示先不考虑输入图片例子维度, 将上一个输出结果展平.

In [41]:
# [n_samples, 3, 3, 64]-->>[n_samples, 3*3*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 2*2*64])

此时weight_variable的shape输入就是第二个卷积层展平了的输出大小: 2x2x64， 后面的输出size为隐藏层神经元数目，我们继续扩大，定为256

In [42]:
W_fc1 = weight_variable([2*2*64, 256])
b_fc1 = bias_variable([256])

然后将展平后的h_pool2_flat与本层的W_fc1相乘（注意这个时候不是卷积了）

In [43]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

如果我们考虑过拟合问题，可以加一个dropout的处理

In [44]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

接下来我们就可以进行最后一层的构建了，好激动啊, 输入是256，最后的输出是10个 (因为mnist数据集就是[0-9]十个类)，prediction就是我们最后的预测值

In [45]:
W_fc2 = weight_variable([256, 10])
b_fc2 = bias_variable([10])

然后呢我们用softmax分类器（多分类，输出是各个类的概率）,对我们的输出进行分类

In [46]:
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)

接着呢我们利用交叉熵损失函数来定义我们的cost function

In [47]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))

我们用tf.train.AdamOptimizer()作为我们的优化器进行优化，使我们的cross_entropy最小

In [48]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

定义Session,初始化变量

In [49]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

好啦接着就是训练数据啦，我们假定训练1000步，每50步输出一下准确率， 注意sess.run()时记得要用feed_dict给我们的众多 placeholder 喂数据哦.

In [50]:
# 计算精确度
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

In [51]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        print compute_accuracy(X_test, y_test)

0.125926
0.138889
0.157407
0.331481
0.52037
0.674074
0.733333
0.746296
0.772222
0.787037
0.82963
0.844444
0.853704
0.866667
0.874074
0.890741
0.903704
0.912963
0.92037
0.925926


可以看到准确率慢慢提升并到了92.6%

# Saver 保存和读取

保存时, 首先要建立一个 tf.train.Saver() 用来保存, 提取变量. 再创建一个名为my_net的文件夹, 用这个 saver 来保存变量到这个目录 "my_net/save_net.ckpt".

In [52]:
saver = tf.train.Saver()
save_path = saver.save(sess, 'my_net/save_net.ckpt')
print 'save to path:', save_path

save to path: my_net/save_net.ckpt
